# COMP0124: Multi-Agent Artificial Intelligence

# Group project: Real-time bidding auctions

**Group #7: Oliviero Balbinetti, Mauricio Caballero, Paul Melkert**

Importing libraries.

In [1]:
import os
import random
import numpy as np
import pandas as pd

from glob import glob
from IPython.display import display

Defining general functions.

In [2]:
#Constant bidding strategy.
def constant_bidding(bid):
    return bid
    
#Random bidding strategy.
def random_bidding(lower, upper):
    return random.randint(lower, upper)
    
#Linear bidding strategy.
def linear_bidding(base_bid, pCTR, Init_CTR):
    return int(base_bid*pCTR/Init_CTR)
    
#Winning criterion #1.
def win_auction_1(bid, payprice):
    return bid >= payprice
    
#Winning criterion #2.
def win_auction_2(bid, payprice, Others):
    return ((bid >= payprice) and (bid >= max(Others)))
    
#Bidding strategy parameter eveluation.
def Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                         Pred_CTRs, prm, Dict_alg, key):
    
    empCPM = []
    Bids, Cost, Imps, Clks = 0, 0, 0, 0
    
    #Looping on bid requests.
    for item in range(0, len(Clicks_Prices)):
        bid = 0
        Bids += 1
        pCTR = Pred_CTRs[item]
        
        #Costant strategy.
        if key == 'Constant':
            Criterion = 1
            bid = constant_bidding(prm)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        #Random strategy.
        elif key == 'Random':
            Criterion = 1
            bid = random_bidding(0, prm)
            Win = win_auction_1(bid, Clicks_Prices[item][1])      
            
        #Linear strategy.
        elif key == 'Linear':
            Criterion = 1
            bid = linear_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        #Exception: error.
        else:
            print('[Strategy error]: Wrong bidding strategy name!')
            break
            
        #Computing bid outcome.
        if Win:
            Imps += 1
            Clks += Clicks_Prices[item][0]
            
            if Criterion == 1:
                Cost += Clicks_Prices[item][1]/1000
                empCPM.append(Clicks_Prices[item][1])
            else:
                Cost += max(Clicks_Prices[item][1],max(others))/1000
                empCPM.append(max(Clicks_Prices[item][1],max(others)))
                
        #Stopping condition.
        if Cost > Budget:
            break
            
    #Reporting results.
    if Imps != 0: empCTR = Clks/Imps
    else: empCTR = 0
    
    avgCPM = np.mean(empCPM)
    if Clks != 0: avgCPC = np.sum(empCPM)/(Clks*1000)
    else: avgCPC = 0
    
    Output =  key + ',' + str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
              + str(Bids) + ',' + str(Imps) + ',' + str(Clks) + ','\
              + str(empCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
    
    return Output
    
#Bidding strategy simulation.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, Pred_CTRs,
                     file, Dict_alg, key):
    parameters = Dict_alg[key]
    
    for prm in parameters:
        Output = Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                                      Pred_CTRs, prm, Dict_alg, key)
        file.write(Output + '\n')

Importing feature engineering data.

In [3]:
#Defining directory.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Feature engineering'

#Importing data.
os.chdir(Path)
Datasets_features = {}
for Filename in glob('*.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Train_ftr = Datasets_features['Train']
Feature_map = Datasets_features['Feature_Map']
Validation_ftr = Datasets_features['Validation']

Importing predicted CTR data.

In [4]:
#Defining directory.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Predicted CTR'

#Importing data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
LR_weights = Datasets_CTR['Logistic regression weights']
Validation_CTR = Datasets_CTR['CTR estimation (Validation)']

Training bidding strategies.

In [5]:
#Training bidding strategies.
Budget = 6250
random.seed(10)

Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Parameter optimization'

#Defining general variables.
Pred_CTRs = []
Clicks_Prices = []

#Initialising parameters.
Prm_cost = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                           np.arange(100, 301, 10)), axis=0)
Prm_rand = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                           np.arange(100, 501, 10)), axis=0)
Prm_lin = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                          np.arange(100, 400, 10), np.arange(400, 800, 50)),
                          axis=0)

Dict_alg = {'Constant': Prm_cost, 'Random': Prm_rand, 'Linear': Prm_lin}

#Initialising values.
Init_CTR = sum(Train_ftr['click'])/len(Train_ftr)
Init_eCPC = sum(Train_ftr['payprice'])/(sum(Train_ftr['click'])*1000)

Pred_CTRs = list(Validation_CTR['pCTR'].values)
Clicks_Prices = list(zip(Validation_ftr['click'], Validation_ftr['payprice']))

#Training and saving results.
os.chdir(Path)

print('Training procedures:\n')
Header = 'algorithm,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC'

for key in Dict_alg.keys():
    Filename = key + '.csv'
    
    with open(Filename, 'w') as file:
        file.write(Header + '\n')
        Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                         Pred_CTRs, file, Dict_alg, key)
        
    print('• [File %s]: Process completed!' %Filename)

Training procedures:

• [File Constant.csv]: Process completed!
• [File Random.csv]: Process completed!
• [File Linear.csv]: Process completed!


Homogeneous multi agents scenario.

In [6]:
#Defining general variables.
print('Homogeneous multi agents scenario:\n')

random.seed(10)
N_agents = np.arange(50, 101)
Parameters = np.concatenate((np.arange(10, 100, 10),np.arange(100, 501, 50)), axis=0)

Filename = 'Multi agents' + '.csv'
Header_MA = 'algorithm,agents,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC'

#Saving multi agents data.
with open(Filename, 'w') as file:
    file.write(Header_MA + '\n')
    
    #Looping on agents.
    for n in N_agents:
        
        #Looping on parameters.
        for prm in Parameters:
            empCPM = []
            Bids, Cost, Imps, Clks = 0, 0, 0, 0
            Output = 'Random (MA)' + ',' + str(n) + ','
            
            #Looping on bid requests.
            for item in range(0, len(Clicks_Prices)):
                bid = 0
                Bids += 1
                Others = []
                
                #Running auction.
                Criterion = 2
                bid = random_bidding(0, prm)
                for i in range(0, n):
                    Others.append(random_bidding(0, prm))
                Win = win_auction_2(bid, Clicks_Prices[item][1], Others)
                
                #Computing outcome.
                if Win:
                    Imps += 1
                    Clks += Clicks_Prices[item][0]
                    Cost += max(Clicks_Prices[item][1],max(Others))/1000
                    empCPM.append(max(Clicks_Prices[item][1],max(Others)))
                    
                #Stopping condition.
                if Cost > Budget:
                    break
                    
            #Reporting results.
            avgCPM = np.mean(empCPM)
            if Imps != 0: empCTR = Clks/Imps
            else: empCTR = 0
            if Clks != 0: avgCPC = np.sum(empCPM)/(Clks*1000)
            else: avgCPC = 0
            
            Output +=  str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
                       + str(Bids) + ',' + str(Imps) + ',' + str(Clks) + ','\
                       + str(empCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
            
            file.write(Output + '\n')
        print('• [Agents: %d]: Process completed!' %n)

Homogeneous multi agents scenario:

• [Agents: 50]: Process completed!
• [Agents: 51]: Process completed!
• [Agents: 52]: Process completed!
• [Agents: 53]: Process completed!
• [Agents: 54]: Process completed!
• [Agents: 55]: Process completed!
• [Agents: 56]: Process completed!
• [Agents: 57]: Process completed!
• [Agents: 58]: Process completed!
• [Agents: 59]: Process completed!
• [Agents: 60]: Process completed!
• [Agents: 61]: Process completed!
• [Agents: 62]: Process completed!
• [Agents: 63]: Process completed!
• [Agents: 64]: Process completed!
• [Agents: 65]: Process completed!
• [Agents: 66]: Process completed!
• [Agents: 67]: Process completed!
• [Agents: 68]: Process completed!
• [Agents: 69]: Process completed!
• [Agents: 70]: Process completed!
• [Agents: 71]: Process completed!
• [Agents: 72]: Process completed!
• [Agents: 73]: Process completed!
• [Agents: 74]: Process completed!
• [Agents: 75]: Process completed!
• [Agents: 76]: Process completed!
• [Agents: 77]: Pro

Model performance evaluation.

In [10]:
#Display model best performance.
print('Best performance and parameters selection for bidding strategy:\n')

#Importing data.
Datasets_parameter = {}

for Filename in glob('*.csv'):
    Datasets_parameter[Filename[:-4]] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Linear = Datasets_parameter['Linear']
Random = Datasets_parameter['Random']
Constant = Datasets_parameter['Constant']
Multi_agents = Datasets_parameter['Multi agents']

#Defining best performances.
List = []
for key in ['Constant','Random','Linear']:
    Data = Datasets_parameter[key]
    
    #Selecting by budget.
    Data = Data[Data['spend'] >= Budget]
    
    #Selecting by clicks.
    Max_clk = max(Data['click'].values)
    Data = Data[Data['click'] == Max_clk]
    
    #Selecting by CTR.
    if len(Data) > 1:
        Max_CTR = max(Data['empCTR'].values)
        Data = Data[Data['empCTR'] == Max_CTR]
        
    #Selecting by avgCPM.
    if len(Data) > 1:
        Min_CPM = min(Data['avgCPM'].values)
        Data = Data[Data['avgCPM'] == Min_CPM]
        
    #Selecting by avgCPC.
    if len(Data) > 1:
        Min_CPC = min(Data['avgCPC'].values)
        Data = Data[Data['avgCPC'] == Min_CPC]
        
    List.append(Data.values[0])
    
#Multi agent scenario.
List_MA = []
for N, Data in Multi_agents.groupby(['agents'], axis=0):
    
    #Selecting by budget.
    #Data = Data[Data['spend'] >= Budget]
    
    #Selecting by clicks.
    Max_clk = max(Data['click'].values)
    Data = Data[Data['click'] == Max_clk]
    
    #Selecting by CTR.
    if len(Data) > 1:
        Max_CTR = max(Data['empCTR'].values)
        Data = Data[Data['empCTR'] == Max_CTR]
        
    #Selecting by avgCPM.
    if len(Data) > 1:
        Min_CPM = min(Data['avgCPM'].values)
        Data = Data[Data['avgCPM'] == Min_CPM]
        
    #Selecting by avgCPC.
    if len(Data) > 1:
        Min_CPC = min(Data['avgCPC'].values)
        Data = Data[Data['avgCPC'] == Min_CPC]
        
    List_MA.append(Data.values[0])
    
#Printing results.
print('• Single agent scenario:')
df = pd.DataFrame(List, columns = Header.split(','))
display(df)

print('\n• Multi agent scenario:')
df = pd.DataFrame(List_MA, columns = Header_MA.split(','))
display(df)

Best performance and parameters selection for bidding strategy:

• Single agent scenario:


,algorithm,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC
0,Constant,85,6250,6250.001,196053,136113,66,0.000485,45.917737,94.696985
1,Random,150,6250,6250.084,261691,137557,72,0.000523,45.436321,86.806722
2,Linear,100,6250,6250.039,301401,122309,155,0.001267,51.100401,40.322832



• Multi agent scenario:


,algorithm,agents,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC
0,Random (MA),50,300,6250,1874.150,303925,6477,8,0.001235,289.354639,234.268750
1,Random (MA),51,450,6250,2715.697,303925,6263,10,0.001597,433.609612,271.569700
2,Random (MA),52,200,6250,1168.628,303925,6040,7,0.001159,193.481457,166.946857
3,Random (MA),53,150,6250,840.884,303925,5781,7,0.001211,145.456495,120.126286
4,Random (MA),54,90,6250,471.708,303925,5377,7,0.001302,87.726985,67.386857
5,Random (MA),55,450,6250,2524.357,303925,5801,11,0.001896,435.158938,229.487000
6,Random (MA),56,450,6250,2482.490,303925,5701,9,0.001579,435.448167,275.832222
7,Random (MA),57,500,6250,2707.997,303925,5594,6,0.001073,484.089560,451.332833
8,Random (MA),58,100,6250,503.591,303925,5161,6,0.001163,97.576245,83.931833
9,Random (MA),59,400,6250,2148.184,303925,5539,8,0.001444,387.828850,268.523000
